In [1]:
import cv2
import mediapipe as mp
import time
import math
import numpy as np
import pyautogui
import ctypes

In [2]:
class handDetector():
    
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands()
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]
        
        
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
#                 if draw:
#                     cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
                xmin, xmax = min(xList), max(xList)
                ymin, ymax = min(yList), max(yList)
                bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                  (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

            # totalFingers = fingers.count(1)

        return fingers
    
    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
            length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]


In [5]:
wScr,hScr= pyautogui.size()
detector=handDetector(maxHands=1)
wCam=640
hCam=480
frameR=100
smoothening=10
plocX,plocY=0,0
clocX,clocY=0,0
#mouse = Controller()
pTime=0
cTime=0
count=0
SetCursorPos = ctypes.windll.user32.SetCursorPos
mouse_event = ctypes.windll.user32.mouse_event
cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
while cap.isOpened():
    success,img=cap.read()
    img = cv2.flip(img, 1)
    img=detector.findHands(img)
    lmList,bbox=detector.findPosition(img)
#     if len(lmList)!=0:
# #         print(lmList[4])
#         length,img,A=detector.findDistance(8,12,img)
#     cv2.putText(img,str(int(length)),(540,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),3)

    cv2.rectangle(img,(frameR,frameR),(wCam-frameR,hCam-frameR),(255,0,255),2)

    if len(lmList)!=0:
        x1,y1=lmList[8][1:]
        x2,y2=lmList[12][1:]
        x4,y4=lmList[4][1:]
        x3,y3=lmList[3][1:]
        x2,y2=lmList[2][1:]
        x16,y16=lmList[16][1:]
        x20,y20=lmList[20][1:]
        x7,y7=lmList[7][1:]
        x11,y11=lmList[11][1:]
        #print(x1,y1,x2,y2)
        fingers=detector.fingersUp()
#         print(fingers)

        if fingers[1]==1 and fingers[2]==0:
        #moving mode
            x3=np.interp(x1,(frameR,wCam-frameR),(0,wScr))
            y3=np.interp(y1,(frameR,hCam-frameR),(0,hScr))
            clocX=plocX+(x3-plocX)/smoothening
            clocY=plocY+(y3-plocY)/smoothening
            
            SetCursorPos(int(clocX),int(clocY))
            #pyautogui.moveTo(int(clocX),int(clocY))
            
            cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
            plocX,plocY=clocX,clocY
            
            if fingers[1]==1 and fingers[4]==1:
                pyautogui.click(button='right',duration=0.25)
            
            
                
        if fingers[1]==1 and fingers[2]==1:
            length,img,line_info=detector.findDistance(8,12,img)
            if length<32:
                cv2.circle(img,(line_info[4],line_info[5]),15,(0,255,0),cv2.FILLED)
                #mouse.click()
                pyautogui.click(duration=0.15)
                
        if fingers[4]==1:
            time.sleep(0.3)
            mouse_event(2,0,0,0,0)
            
        if fingers[2]==1 and fingers[1]==0:
            mouse_event(4,0,0,0,0)
        
        
        
            
#             else:
#                 pyautogui.scroll(-150)
        
        elif y4<y1 and y4<y2 and y4<y16 and y4<y20:
            pyautogui.scroll(150)
            
        elif y4>y1 and y4>y2 and y4>y16 and y4>y20:
            pyautogui.scroll(-150)
                
                
        #elif fingers[1]==1 and fingers[2]==1 and fingers[3]==1:
           
            
            
    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),3)
    cv2.imshow('Image',img)
    k = cv2.waitKey(1) & 0xFF    
    if(k == 27):
        break                
cap.release()
cv2.destroyAllWindows()